In [ ]:
from pathlib import Path
from dotenv import load_dotenv
import warnings
import pandas as pd
import hopsworks
import datetime
import os

import openmeteo_requests

import requests_cache
from retry_requests import retry

warnings.filterwarnings("ignore")


In [ ]:
load_dotenv("../.env", override=True)
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
if HOPSWORKS_API_KEY is None:
    raise RuntimeError(
        "HOPSWORKS_API_KEY not found. "
    )


project = hopsworks.login()
fs = project.get_feature_store()
today = datetime.date.today()

In [ ]:


#weather data
#taken from openmeteo

def hourly_weather_forecast(latitude, longitude, forecast_days):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = 36002)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)
    
    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
    	"latitude": latitude,
    	"longitude": longitude,
        "hourly": ["temperature_2m", "shortwave_radiation", "cloud_cover", "wind_speed_10m", "precipitation"],
        "forecast_days":forecast_days,
    }
    responses = openmeteo.weather_api(url, params=params)
    
    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation: {response.Elevation()} m asl")
    print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")
    
    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    shortwave_radiation = hourly.Variables(1).ValuesAsNumpy()
    cloud_cover = hourly.Variables(2).ValuesAsNumpy()
    wind_speed_10m = hourly.Variables(3).ValuesAsNumpy()
    precipitation = hourly.Variables(4).ValuesAsNumpy()
   
    
    hourly_data = {"datetime": pd.date_range(
    	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    	freq = pd.Timedelta(seconds = hourly.Interval()),
    	inclusive = "left"
    )}
    
    hourly_data["temperature_2m"] = temperature_2m
    hourly_data["shortwave_radiation"] = shortwave_radiation
    hourly_data["cloud_cover"] = cloud_cover
    hourly_data["wind_speed_10m"] = wind_speed_10m
    hourly_data["precipitation"] = precipitation
    
    
    hourly_data_df = pd.DataFrame(data = hourly_data)
    hourly_data_df = hourly_data_df.dropna(how="all")

    
    hourly_data_df["datetime"] = hourly_data_df["datetime"].dt.tz_convert(None)
    print(f"Fetched {len(hourly_data_df)} hourly rows")
    return hourly_data_df


AREAS = {
    #"SE1": {"lat": 65.58, "lon": 22.15},
    #"SE2": {"lat": 62.39, "lon": 17.30},
    "SE3": {"lat": 59.33, "lon": 18.06},
    #SE4": {"lat": 55.60, "lon": 13.00},
}

weather_dfs = []

for area, coords in AREAS.items():
    df_weather = hourly_weather_forecast(
        latitude=coords["lat"],
        longitude=coords["lon"],
        forecast_days = 7
    )
    df_weather["area"] = area
    weather_dfs.append(df_weather)

weather_all = pd.concat(weather_dfs).reset_index(drop=True)


In [ ]:
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)
weather_fg.insert(weather_all)
